In [ ]:
# !pip uninstall mediapipe opencv-python==4.5.5.64
!pip uninstall mediapipe opencv-python==0.9.3.0
# !pip uninstall opencv-python
# !pip uninstall opencv-contrib-python
# !pip uninstall opencv-contrib-python-headless
# !pip uninstall -U tensorflow-estimator==2.6.0
# !pip uninstall tensorflow --upgrade

In [ ]:
!pip install mediapipe
# !pip install mediapipe

In [68]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd
import math
from statistics import mean
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [79]:
# Calculate angle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


# Calculate distance
def calculate_distance(a,b):
    x1, y1 = a
    x2, y2 = b
    
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    return distance


# Calculate velocity
def calculate_velocity(distance, time):
    velocity = distance / time
    
    return velocity


# Determine good/bad rep
def determine_rep(temp_frames):
    return temp_frames

In [54]:
# Create CSV file
csvfile = open('bicep_curl.csv', 'w', newline='')
csvwriter = csv.writer(csvfile)
# csvwriter.writerow(['shoulder_x', 'shoulder_y', 'elbow_x', 'elbow_y', 'wrist_x', 'wrist_y'])
csvwriter.writerow(['shoulder', 'elbow', 'wrist'])

22

In [84]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "contract"
            if angle < 30 and stage =='contract':
                stage="extend"
                counter +=1
            
            # Calculate rep score
                # Log all frames to temp_frames Y
                # Call function(rep, temp_frames)
                    # Elbow inline with shoulder for all frames? Else: Elbow = Bad, Break
                    # Velocity below threshold for all frames? Else: Velocity = Bad, Break
                    # Range within threshold for all frames? Range: Rep = Bad, Break
                # Log elbow, velocity, range feedback to feedback.csv 
                # Return "Bad", score -= 1 && clear temp_frames
       
            # Initiate temporary frame storage for rep
            temp_frames = []
    
            # Calculate rep score
            temp_frames = [shoulder, elbow, wrist]
            print(temp_frames)
            # determine_rep(temp_frames)
            
            # Save to CSV
            csvwriter.writerow([shoulder, elbow, wrist])


        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    csvfile.close()
    cap.release()
    cv2.destroyAllWindows()


[[0.6501689553260803, 0.3869238793849945], [0.6815844178199768, 0.6198149919509888], [0.6959806680679321, 0.8113176226615906]]
[[0.6501988768577576, 0.38738760352134705], [0.6817842721939087, 0.6200549006462097], [0.6955696940422058, 0.8122197389602661]]
[[0.6502472162246704, 0.38742077350616455], [0.6818692088127136, 0.620052695274353], [0.6952131390571594, 0.812357485294342]]
[[0.6502718925476074, 0.3874221444129944], [0.6818827390670776, 0.6199731826782227], [0.694850742816925, 0.812362015247345]]
[[0.650450587272644, 0.38751518726348877], [0.6819058656692505, 0.6199700236320496], [0.6947037577629089, 0.8123677968978882]]
[[0.6506223082542419, 0.3874622881412506], [0.6819857954978943, 0.6195277571678162], [0.6945446729660034, 0.8117417097091675]]
[[0.6508259773254395, 0.3874630630016327], [0.682063639163971, 0.619153618812561], [0.6944200992584229, 0.8113477826118469]]
[[0.651420533657074, 0.38746222853660583], [0.6824327707290649, 0.6191160082817078], [0.694411039352417, 0.81134784

[[0.6489042639732361, 0.3766718804836273], [0.6908971071243286, 0.6216081976890564], [0.6964188814163208, 0.391238808631897]]
[[0.6476532220840454, 0.374942421913147], [0.6916699409484863, 0.6217179894447327], [0.6931380033493042, 0.39604899287223816]]
[[0.6470778584480286, 0.3738982081413269], [0.6918155550956726, 0.621724009513855], [0.6927605271339417, 0.40238627791404724]]
[[0.6499868035316467, 0.37294644117355347], [0.6913607120513916, 0.6218080520629883], [0.6943716406822205, 0.4266642928123474]]
[[0.6558716893196106, 0.3776366114616394], [0.6904674768447876, 0.6257377862930298], [0.6962513327598572, 0.4531412422657013]]
[[0.6559634208679199, 0.3810226321220398], [0.6910188794136047, 0.6266504526138306], [0.7045865058898926, 0.5746458172798157]]
[[0.6559558510780334, 0.385710746049881], [0.6917670369148254, 0.6340948939323425], [0.7091137170791626, 0.6259256601333618]]
[[0.6559223532676697, 0.39040711522102356], [0.6914559602737427, 0.6321266889572144], [0.71729576587677, 0.75895

[[0.6966374516487122, 0.3847498893737793], [0.721645712852478, 0.6348761320114136], [0.7458133697509766, 0.8672016263008118]]
[[0.7027425169944763, 0.37833210825920105], [0.7340942621231079, 0.6351386308670044], [0.7640432119369507, 0.8714686036109924]]
[[0.7064436078071594, 0.373042494058609], [0.7407664060592651, 0.6352043747901917], [0.7700260877609253, 0.8799825310707092]]
[[0.7039216756820679, 0.34801098704338074], [0.7503868341445923, 0.6347078084945679], [0.7831934690475464, 0.9090240597724915]]
[[0.7026460766792297, 0.3248043358325958], [0.7526360750198364, 0.6177772879600525], [0.7839884161949158, 0.896468460559845]]
[[0.7024278044700623, 0.27872514724731445], [0.7645142078399658, 0.6004390120506287], [0.7874311804771423, 0.9035369157791138]]
[[0.6962536573410034, 0.2539871633052826], [0.7627735733985901, 0.5782156586647034], [0.7907752394676208, 0.8948280811309814]]
[[0.6971514821052551, 0.2235516458749771], [0.7785762548446655, 0.55116206407547], [0.8119374513626099, 0.88857

[[0.8643830418586731, 0.22183728218078613], [1.0192883014678955, 0.7136433124542236], [1.169535756111145, 1.2734242677688599]]
[[0.8643637895584106, 0.22266672551631927], [1.0129729509353638, 0.7140516042709351], [1.1689679622650146, 1.2742863893508911]]
[[0.8643280267715454, 0.2226925790309906], [1.0102829933166504, 0.712047278881073], [1.1749643087387085, 1.2741646766662598]]
[[0.8638826608657837, 0.22196918725967407], [1.005456805229187, 0.7106301188468933], [1.1800910234451294, 1.273714303970337]]
[[0.8638613820075989, 0.22176557779312134], [1.0062967538833618, 0.7086345553398132], [1.1706937551498413, 1.2658921480178833]]
[[0.863005518913269, 0.22142301499843597], [1.0062756538391113, 0.6981685757637024], [1.1696274280548096, 1.2552682161331177]]
[[0.8626500368118286, 0.22013549506664276], [1.002148985862732, 0.6962657570838928], [1.1573340892791748, 1.2552059888839722]]
[[0.8627799153327942, 0.21946145594120026], [0.9968241453170776, 0.6966431736946106], [1.146824598312378, 1.255

[[0.9240215420722961, 0.2147798389196396], [1.0975675582885742, 0.8105558156967163], [1.1665425300598145, 1.282058835029602]]
[[0.9213069081306458, 0.21495667099952698], [1.0994479656219482, 0.8083957433700562], [1.1652034521102905, 1.2694990634918213]]
[[0.9167574644088745, 0.20959845185279846], [1.0919941663742065, 0.8038826584815979], [1.1631641387939453, 1.2557666301727295]]
[[0.9112180471420288, 0.20961059629917145], [1.0914608240127563, 0.7771390080451965], [1.1686798334121704, 1.2175166606903076]]
[[0.8996340036392212, 0.20575226843357086], [1.075626254081726, 0.7465460300445557], [1.166120171546936, 1.202948808670044]]
[[0.8867756724357605, 0.20942875742912292], [1.0192043781280518, 0.718357503414154], [1.138218879699707, 1.2042196989059448]]
[[0.8803447484970093, 0.21599657833576202], [0.9935568571090698, 0.7097004652023315], [1.11716628074646, 1.2050691843032837]]
[[0.8749682307243347, 0.22069548070430756], [0.9881108999252319, 0.7044103145599365], [1.0942366123199463, 1.2090

[[0.7816647291183472, 0.16114845871925354], [0.9106370806694031, 0.43958237767219543], [0.8774756193161011, 0.7420576214790344]]
[[0.8403881788253784, 0.26930710673332214], [0.9566459655761719, 0.5237947702407837], [0.9030488133430481, 0.841643750667572]]
[[0.8684695959091187, 0.2831474542617798], [0.9517098665237427, 0.5234149098396301], [0.917008101940155, 0.8600905537605286]]
[[0.8409758806228638, 0.3460354208946228], [0.93641197681427, 0.6146120429039001], [0.9484850764274597, 0.8366587162017822]]
[[0.8453486561775208, 0.3508293330669403], [0.9472753405570984, 0.6298776268959045], [0.9835933446884155, 0.858012318611145]]
[[0.8400707244873047, 0.27249765396118164], [0.9678528904914856, 0.613536536693573], [0.9990559816360474, 0.8300281763076782]]
[[0.8505957722663879, 0.2955407500267029], [0.9763766527175903, 0.7021386027336121], [1.061354398727417, 0.9906826019287109]]
[[0.8536895513534546, 0.36491212248802185], [0.9606566429138184, 0.6972392201423645], [1.0318257808685303, 0.94474

[[0.5070727467536926, 0.8307744264602661], [0.6585318446159363, 1.1862633228302002], [0.734679102897644, 1.5063458681106567]]
[[0.5515036582946777, 0.8279946446418762], [0.7048355937004089, 1.2110507488250732], [0.7738611698150635, 1.513399362564087]]
[[0.5764352083206177, 0.8274075984954834], [0.7135993242263794, 1.2316968441009521], [0.7755070924758911, 1.5280826091766357]]
[[0.6111425161361694, 0.8280567526817322], [0.7561674118041992, 1.2562825679779053], [0.7872678637504578, 1.5801329612731934]]
[[0.6395272612571716, 0.8269444108009338], [0.8014310598373413, 1.2638591527938843], [0.8168187141418457, 1.6090967655181885]]
[[0.6551464796066284, 0.826160192489624], [0.8265422582626343, 1.2626335620880127], [0.8609800338745117, 1.610050082206726]]
[[0.6791887879371643, 0.8102256655693054], [0.8866069912910461, 1.2478989362716675], [0.9586639404296875, 1.591981291770935]]
[[0.7067115902900696, 0.7935498952865601], [0.9011491537094116, 1.2439496517181396], [0.9602863192558289, 1.58470475

In [ ]:
# Read CSV file
df = pd.read_csv('bicep_curl.csv')
print(df)

In [78]:
# Define the ideal form criteria and the scoring system
ELBOW_POSITION_THRESHOLD = 0.2  # Maximum allowed distance between elbow and shoulder keypoints
FULL_RANGE_OF_MOTION_ANGLE = 90  # Ideal angle between shoulder, elbow, and wrist keypoints
SMOOTHNESS_THRESHOLD = 10  # Maximum allowed change in velocity between wrist keypoints

PERFECT_FORM_SCORE = 10  # Maximum score for perfect form
ELBOW_POSITION_PENALTY = 1  # Penalty per unit distance above threshold
RANGE_OF_MOTION_PENALTY = 1  # Penalty per unit angle deviation from ideal
SMOOTHNESS_PENALTY = 1  # Penalty per unit velocity change above threshold

# Calculate the measurements for each repetition
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

elbow_distance = calculate_distance(elbow, shoulder)
rom_angle = calculate_angle(shoulder, elbow, wrist)
# velocity = calculate_velocity(wrist_history)

# Compare the measurements to the ideal form criteria and assign a score
elbow_penalty = max(0, (elbow_distance - ELBOW_POSITION_THRESHOLD) / ELBOW_POSITION_PENALTY)
rom_penalty = max(0, (abs(rom_angle - FULL_RANGE_OF_MOTION_ANGLE)) / RANGE_OF_MOTION_PENALTY)
# smoothness_penalty = max(0, (velocity - SMOOTHNESS_THRESHOLD) / SMOOTHNESS_PENALTY)

# form_score = PERFECT_FORM_SCORE - elbow_penalty - rom_penalty #- smoothness_penalty
form_score = mean((elbow_penalty, rom_penalty)) #- smoothness_penalty)

# Keep a running total of the scores for each repetition
total_score += form_score

# Report the average score at the end of the exercise
num_repetitions = counter
# average_score = total_score / num_repetitions
average_score = total_score

print(f"Average score for {num_repetitions} = {average_score}")


Average score for 0 = 159.24762928464557
